### Data read in and Train-Test split

In [1]:
import scipy.io as sio
import numpy as np

matfile = sio.loadmat('./Sogou_data/Sogou_webpage.mat')
label = matfile['doclabel']
feature = matfile['wordMat']

data = np.concatenate((feature,label), axis=1)
np.random.shuffle(data)

train_data = data[:int(0.6*data.shape[0]),:]
valid_data = data[int(0.6*data.shape[0]):int(0.8*data.shape[0]),:]
test_data = data[int(0.8*data.shape[0]):,:]

train = {'feature':None,'label':None}
valid = {'feature':None,'label':None}
test = {'feature':None,'label':None}

for pair in [[train_data,train],[valid_data,valid],[test_data,test]]:
    pair[1]['feature'] = pair[0][:,:-1]
    pair[1]['label'] = pair[0][:,-1]
    
for dataset in [train,valid,test]:
    print (dataset['feature'].shape,dataset['label'].shape)

(8640, 1200) (8640,)
(2880, 1200) (2880,)
(2880, 1200) (2880,)
